In [1]:
import pandas as pd
import re

In [2]:
# turn the whole excel file into csv files thanks to Python

excel_path = 'data/resume/data_resume.xlsx'

# Read the whole files and the associated tabs
xls_tab = pd.read_excel(excel_path, sheet_name=None)

# Assign tab value to different variable
me = xls_tab['Me']
locations = xls_tab['Locations']
experiences = xls_tab['Experiences']
educations = xls_tab['Educations']
contracts = xls_tab['Contracts']
skills = xls_tab['Skills']
companies = xls_tab['Companies']
skill_experiences = xls_tab['Skill_experiences']
degrees = xls_tab['Degrees']
certifications = xls_tab['Certification']
schools = xls_tab['Schools']
leisures = xls_tab['Leisures']
languages = xls_tab['Languages']
qualities = xls_tab['Qualities']
category_skills = xls_tab['Category_skills']

In [3]:
# Fill the NaN dates to current timestamp

# Turn all the NaT in a current date and normalize it back to a normal date
now_normalized = pd.Timestamp.now().normalize()
now_month = now_normalized.strftime('%B')
now_year = now_normalized.year

def set_current_time_nan_date(dates):
    return pd.to_datetime(dates).fillna(pd.Timestamp.now().normalize())

In [25]:
# I normalize all the date in the experiences table
experiences['start_date'] = pd.to_datetime(experiences['start_date'])
experiences['end_date'] = pd.to_datetime(experiences['end_date'])

# Empty the null dates with the current timestamp normalized
experiences['start_date'] = set_current_time_nan_date(experiences['start_date'])
experiences['end_date'] = set_current_time_nan_date(experiences['end_date'])

experiences['start_month'] = pd.to_datetime(experiences['start_date']).dt.month_name()
experiences['start_year'] = pd.to_datetime(experiences['start_date']).dt.year
experiences['end_month'] = pd.to_datetime(experiences['end_date']).dt.month_name()
experiences['end_year'] = pd.to_datetime(experiences['end_date']).dt.year

In [26]:
# Just the year is stored in the education section, no need to convert in datetime
# But need to convert in 'int' instead of a 'float' to remove the ',0'
educations['year_end'] = educations['year_end'].astype('Int64')
educations['degree_id'] = educations['degree_id'].astype('Int64')

In [27]:
# Search a words in a dataf
def search_word_column_field(dataframe):
    return [col for col in dataframe.columns if re.search('_id$|^id_|id', col)]

In [7]:
# Put the current timestamp to 'Present'

PERSONAL INFORMATIONS SECTION

In [31]:
# personal infos
me_locations = me.merge(locations, 
                        left_on='location_id',
                        right_on='id',
                        suffixes=('', '_location'))

In [9]:
name = f'{me_locations['first_name'].iloc[0]} {me_locations['last_name'].iloc[0]}\n'
driving_licence = 'Yes' if me_locations.loc[0, 'driving_licence'] == 1 else 'No'
driving_licence = f'Driving licence : {driving_licence}\n'
phone = f'📞 +33{me_locations['phone'].iloc[0]}\n'
email = f'✉️ {me_locations['email'].iloc[0]}\n'
localisation  = f'📍 {me_locations['city'].iloc[0]}, {me_locations['country'].iloc[0]}'
personal_info = name + driving_licence + phone + email + localisation + '\n\n'

QUALITIES SECTION

In [10]:
# display the qualities 
quality_str = 'QUALITIES \n'
quality_str += '----------------------------------------------------------------------------- \n\n'
quality_str += ' | '.join(qualities['name'])
quality_str += '\n\n'

SKILLS & CATEGORIES

In [11]:
# retrieve the infos from category and skills
skills_categories_merged = skills.merge(category_skills, 
                                        left_on='category_id',
                                       right_on='id',
                                       suffixes=('', '_category'))

In [12]:
skills_str = 'SKILLS \n'
skills_str += '----------------------------------------------------------------------------- \n\n'

# Create a new column to concatenate all the skills with their level
skills_categories_merged['skill_with_level'] = skills_categories_merged.apply(
    lambda row: f'{row['name']} ({row['level']})', axis=1
)

# group by category to display in a better view
grouped_skills_category = skills_categories_merged.groupby('name_category', as_index = False).agg({
    'skill_with_level': lambda x: ', '.join(x)
})

for index, skill in grouped_skills_category.iterrows():
    skills_str += f'{skill['name_category']} : {skill['skill_with_level']}\n\n'
skills_str += '\n'

EXPERIENCES SECTION

In [13]:
# Merged all the relevant tables to display the experiences
skills_skillexperiences = skill_experiences.merge(skills_categories_merged, 
                                       left_on='skill_id', 
                                       right_on='id',
                                                 suffixes=('','_skill'))
experience_skills = skills_skillexperiences.merge(experiences, 
                                                  left_on='experience_id', 
                                                  right_on='id', 
                                                  suffixes=('','_experience'))
experience_skills_companies = experience_skills.merge(companies, 
                                                      left_on='company_id', 
                                                      right_on='id',
                                                     suffixes=('', '_company'))
experience_skills_companies_contracts = experience_skills_companies.merge(contracts,
                                                               left_on ='contract_id',
                                                               right_on = 'id',
                                                               suffixes=('', '_contract'))
experiences_merged = experience_skills_companies_contracts.merge(locations, 
                                                left_on='location_id', 
                                                right_on='id',
                                               suffixes=('', '_location'))

In [14]:

fields_experiences_contain_id = search_word_column_field(experiences_merged)
experiences_merged = experiences_merged.drop(columns=fields_experiences_contain_id)

In [15]:
# rename the fields that are ambiguous
experiences_merged = experiences_merged.rename(columns={'name' : 'skill'})
experiences_merged.head()

,skill,level,name_category,skill_with_level,start_date,end_date,job_title,job_desc,start_month,start_year,end_month,end_year,name_company,industry,type,city,country
0,Management,Intermediate,IT project and Product Management,Management (Intermediate),2015-04-01,2015-09-01,tourism production assistant,"In collaboration with the head ""Group' departm...",April,2015,September,2015,Magic Travel Group LLC,Tourism,Work placement,NYC,USA
1,Marketing,Intermediate,Marketing and Communication,Marketing (Intermediate),2015-04-01,2015-09-01,tourism production assistant,"In collaboration with the head ""Group' departm...",April,2015,September,2015,Magic Travel Group LLC,Tourism,Work placement,NYC,USA
2,Project Management,Intermediate,IT project and Product Management,Project Management (Intermediate),2015-04-01,2015-09-01,tourism production assistant,"In collaboration with the head ""Group' departm...",April,2015,September,2015,Magic Travel Group LLC,Tourism,Work placement,NYC,USA
3,Customer Service,Intermediate,Marketing and Communication,Customer Service (Intermediate),2015-04-01,2015-09-01,tourism production assistant,"In collaboration with the head ""Group' departm...",April,2015,September,2015,Magic Travel Group LLC,Tourism,Work placement,NYC,USA
4,Communication,Advanced,Marketing and Communication,Communication (Advanced),2015-04-01,2015-09-01,tourism production assistant,"In collaboration with the head ""Group' departm...",April,2015,September,2015,Magic Travel Group LLC,Tourism,Work placement,NYC,USA


In [16]:
# Display all the experiences

# All rows are duplicated because for each skill associated with an experience
# Objective : group all the experiences and concatenate all skills per experience
# That's why I filter those columns to not taking in account when grouping
columns_to_take = [col for col in experiences_merged.columns if col not in[
    'skill', 'name_category', 'skill_with_level', 'level']]

grouped_experiences = experiences_merged.groupby(columns_to_take, as_index=False).agg({
    'skill': lambda x: ', '.join(sorted(set(x)))  # Combine skill names into one string
})
grouped_experiences = grouped_experiences.sort_values('start_date', ascending=False)
grouped_experiences.head()

,start_date,end_date,job_title,job_desc,start_month,start_year,end_month,end_year,name_company,industry,type,city,country,skill
10,2025-06-01,2025-07-28,Seller in an organic store,I worked again as organic store assitant to ke...,June,2025,July,2025,La Vie Claire,Organic,Short-term contract,Lyon,France,"Communication, Management, Marketing, Project ..."
9,2022-08-01,2025-07-28,"Health therapist : digestion, sleep management...",I built my own business in the health industry...,August,2022,July,2025,Myself,Health,Freelance,Lyon,France,"Communication, Management, Marketing, Project ..."
8,2020-03-01,2021-02-01,Working holiday Visa in Japan,I had a dream coming to Japan for one year thr...,March,2020,February,2021,Myself,Language teaching,Other,Osaka,Japan,"English, French, Spanish"
7,2019-09-01,2019-11-01,Seller in an organic store,I worked as organic shop assistant because I w...,September,2019,November,2019,La Vie Claire,Organic,Short-term contract,Lyon,France,"Communication, Communication marketing, Custom..."
6,2019-01-01,2019-07-01,Angular/java developer,Developing a whole web application for a web-b...,January,2019,July,2019,CGI,SSII,Long-term contract,Lyon,France,"PostgreSQL, Project Management, Python, SQL"


In [17]:
# displaying the experiences as we can see in a resume
experience = 'EXPERIENCES \n'
experience += '----------------------------------------------------------------------------- \n\n'
for index, items in grouped_experiences.iterrows():
    # I check if the date is equal to current timestamp to replace by present
    if items['end_month']==now_month and items['end_year']==now_year:
        items['end_month'], items['end_year'] = 'Present', ''
    
    dates = '📅 {} - {} \n'.format(
        str(items['start_month']) + ' ' + str(items['start_year']), 
        str(items['end_month'])+ ' ' + str(items['end_year']))
    job_title = '** {} '.format(items['job_title'])
    company = f'{items['name_company']} - {items['city']}, {items['country']} ** '
    contract_type = f'({items['type']})\n'
    job_description = '{} \n'.format(items['job_desc'])
    skill_got = 'Skills acquired: {} \n\n'.format(items['skill'])
    experience += dates + job_title + company + contract_type + job_description + skill_got + '\n'

EDUCATION SECTION

In [18]:
# merge with degree and school with location
educations_degree = educations.merge(degrees,
                                     left_on='degree_id', 
                                     right_on='id',
                                     suffixes=('','_degree')) 
educations_degrees_schools = educations_degree.merge(schools,
                                                     left_on='school_id', 
                                                     right_on='id',
                                                     suffixes=('', '_school'))
educations_merged = educations_degrees_schools.merge(locations,
                                                     left_on='location_id',
                                                     right_on='id',
                                                    suffixes=('', '_location'))


In [19]:
# Search and remove the columns with id
fields_educations_contain_id = search_word_column_field(educations_merged)
educations_merged = educations_merged.drop(columns=fields_educations_contain_id)


In [30]:
educations_merged = educations_merged.sort_values('year_start', ascending=False)

In [29]:
# displaying the experiences as we can see in a resume
education_str = 'EDUCATION \n'
education_str += '----------------------------------------------------------------------------- \n\n'
for index, items in educations_merged.iterrows():
    if pd.isna(items['year_end']):
            items['year_end'] = 'Present'
    dates = '📅 {} - {} \n'.format(
        str(items['year_start']), str(items['year_end']))
    degree = '** {} - {} **\n'.format(
        items['type'], 
        items['description'])
    school = '{} - {} - {} - {}\n\n'.format(
        items['name'], 
        items['type_school'], 
        items['city'],
        items['country'])
    education += dates + degree + school + '\n'

LEISURES

In [22]:
# displaying the experiences as we can see in a resume
leisure_str = 'LEISURES \n'
leisure_str += '----------------------------------------------------------------------------- \n\n'
leisure_str += ' | '.join(leisures['name'])
leisure_str += '\n\n'

WHOLE RESUME

In [28]:
whole_resume = personal_info + quality_str + skills_str + experience + education_str + leisure_str
print(whole_resume)

Cédric BERTHEZENE
Driving licence : Yes
📞 +33671927780
✉️ cedric.berthezene@gmail.com
📍 Lyon, France

QUALITIES 
----------------------------------------------------------------------------- 

Autonomous | Flexible | Dynamique | Trustworthy | Persevering | Strong-willed

SKILLS 
----------------------------------------------------------------------------- 

Data analysis and Business Intelligence : Business Strategy (Intermediate), Sales strategy (Intermediate)

Databases and Data Management : Database management (Intermediate), Database queries (Intermediate), Database design  (Intermediate), Jupyter (Intermediate), PostgreSQL (Intermediate)

IT project and Product Management  : Management (Intermediate), Project Management (Intermediate)

Languages : Spanish (Intermediate), English (Advanced)

Marketing and Communication : Marketing Strategy (Intermediate), Marketing Communications (Intermediate), Marketing (Intermediate), Social Media (Intermediate), Customer Service (Intermediate),